# Jupyter-Notebook for EE assignment

### Install the EE API

In [1]:
# conda install -c conda-forge earthengine-api
# conda install folium -c conda-forge

### Import libraries

In [1]:
import ee
import folium

### Authentication/Initialization

In [4]:
# opens browser to request authenticatication token (OAUTH2) and store it.
#ee.Authenticate()

# uses your stored auth token to login to ee.
ee.Initialize()

### Define the subclass of folium.Map

In [5]:
class EngineMap(folium.Map):
    """
    Subclass of folium Map for adding EE images. Here
    self is a folium.Map class instance, and we are adding 
    additional functions to this class type.
    """
    
    def __init__(self, location=None, zoom_start=None, **kwargs):
        # inherit from parent class
        super().__init__(location=location, zoom_start=zoom_start, **kwargs)
    
    
    def add_ee_vector(self, feature, **kwargs):
        """
        Add a google earth engine vector feature to a folium map.
        """
        # create vector feature
        feature = folium.GeoJson(
            data=feature.geometry().getInfo(),
            **kwargs,
        )
        
        # add vector to Map and return self to allow chaining
        self.add_child(feature)
        return self
    
    
    def add_ee_raster(self, image, vis_params={}, **kwargs):
        """
        Add a google earth engine raster layer to a folium map.
        """       
        # handle ImageCollections and Images
        if isinstance(image, ee.ImageCollection):
            ee_image = image.mosaic()

        # get the JSON instructions to show image tiles
        map_id_dict = image.getMapId(vis_params)
        
        # get url of the raster tiles
        tiles = map_id_dict['tile_fetcher'].url_format
        
        # create a folium raster layer and add to self
        raster = folium.raster_layers.TileLayer(
            tiles=tiles,
            attr=kwargs.get("attr", "TODO-attr"),
            name=kwargs.get("name", "test"),
            overlay=True,
            control=True,
        )
        
        # add raster to Map and return self to allow chaining
        self.add_child(raster)
        return self

### Add image from Hansen et. al.

In [18]:
Hansen = ee.Image("UMD/hansen/global_forest_change_2015")

### Draw the map as the tutorial shows

In [36]:
hmap = EngineMap([35, -95], zoom_start=3)
TC = Hansen.select('treecover2000')
lossImage= Hansen.select('loss')
gainImage= Hansen.select('gain')
hmap.add_ee_raster(
    TC.updateMask(TC),
    name='forest cover',
    attr='Alex Jin',
    vis_params={
    'palette': ['000000', '00FF00'],
    'max': 100
    }
)
hmap.add_ee_raster(
    lossImage.updateMask(lossImage),
    name='Loss',
    attr='Alex Jin',
    vis_params={
    'palette': ['FF0000']
    }
)
hmap.add_ee_raster(
    gainImage.updateMask(gainImage),
    name='Gain',
    attr='Alex Jin',
    vis_params={
    'palette': ['0000FF']
    }
)